In [ ]:
CATEGORY = 'artist' # artist or track
TIMESPAN = 'short_term' # short_term, medium_term, long_term
LIMIT = 50 # 1-50

In [ ]:
import spotipy

import requests
import math
import os

with open('token.txt', 'r') as f:
    token = f.read()

sp = spotipy.Spotify(auth=token)

In [ ]:
top_items = None

if CATEGORY == 'artist':
    top_items = sp.current_user_top_artists(limit=LIMIT, time_range=TIMESPAN)
elif CATEGORY == 'track':
    top_items = sp.current_user_top_tracks(limit=LIMIT, time_range=TIMESPAN)

In [ ]:
# extract image url and id from each item and save it to a list
images = []
for item in top_items['items']:
    images.append({
        'id': item['id'],
        'url': item['images'][0]['url']
    })

In [ ]:
# download images
if not os.path.exists('images'):
    os.mkdir('images')

for image in images:
    if os.path.exists(f'images/{image["id"]}.jpg'):
        continue
    r = requests.get(image['url'])
    with open(f'images/{image["id"]}.jpg', 'wb') as f:
        f.write(r.content)

In [ ]:
with open(f'{CATEGORY}s_{TIMESPAN}_size{LIMIT}.html', 'w') as f:
    f.write('<html>')
    f.write('<head>')

    f.write('<style>')
    f.write('body {background-color: #121212;}')
    f.write('table {margin-left: auto; margin-right: auto;}')
    f.write('table, h1 {margin-top: 30px;}')
    f.write('table {margin-bottom: 30px;}')
    f.write('td {padding: 5px;}')
    f.write('img {border: 2px solid #fff;}')
    f.write('img {border-radius: 33%;}')
    f.write('img {transition: 0.5s;}')
    f.write('img:hover {border-radius: 33%;}')
    f.write('img:hover {transition: 1.0s;}')
    f.write('img:hover {transform: scale(3);}')
    f.write('</style>')

    f.write('</head>')
    f.write('<body>')

    CATEGORY_TITLE = CATEGORY.title()
    TIMESPAN_TITLE = TIMESPAN.replace('_', ' ').title()
    f.write(f'<h1 style="text-align: center; color: white; font-family: Consolas;">Your Top {CATEGORY_TITLE}s [{TIMESPAN_TITLE}]</h1>')

    f.write('<table>')
    length_of_line = int(math.sqrt(len(images)))
    for x in range(0, length_of_line):
        f.write('<tr>')
        for y in range(0, length_of_line):
            f.write(f'<td><a href="https://open.spotify.com/{CATEGORY}/{images[x*length_of_line+y]["id"]}"><img src="images/{images[x*length_of_line+y]["id"]}.jpg" width="100" height="100"></a></td>')
        f.write('</tr>')
    f.write('</table>')

    f.write('</body>')
    f.write('</html>')

print(f'Created file: {CATEGORY}s_{TIMESPAN}_size{LIMIT}.html')
